### T2. Exam Question (2nd round) / 기출 (2회차) : E-Commerce Shipping Data
* 성능이 우수한 예측모형을 구축하기 위해서는 적절한 데이터 전처리, 피처엔지니어링, 분류알고리즘, 하이퍼파라미터 튜닝, 모형 앙상블 등이 수반되어야 한다.
* 수험번호.csv파일이 만들어지도록 코드를 제출한다.
* 제출한 모델의 성능은 ROC-AUC형태로 읽어드린다.

In [46]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("./datasets/customer-analytics/Train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Reached.on.Time_Y.N', id_name='ID')

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((8799, 11), (2200, 11), (8799, 2), (2200, 2))

In [47]:
X_train.drop(["Warehouse_block","Mode_of_Shipment", "Gender", "Product_importance"], axis=1,inplace=True)
X_test.drop(["Warehouse_block","Mode_of_Shipment", "Gender", "Product_importance"], axis=1,inplace=True)


In [48]:
X_train_id = X_train.pop('ID')
X_test_id  = X_test.pop('ID')
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)


In [50]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train["Reached.on.Time_Y.N"])
pred = rf.predict(X_test)
pred


array([1, 1, 1, ..., 0, 1, 1], dtype=int64)

In [51]:
submission =pd.DataFrame({"ID":X_test_id,"Reached.on.Time_Y.N":pred})
submission.head()

,ID,Reached.on.Time_Y.N
4732,4733,1
2039,2040,1
5113,5114,1
2360,2361,1
5995,5996,0


In [52]:
submission.to_csv('submission.csv', index=False)

In [53]:
round(rf.score(X_test, y_test['Reached.on.Time_Y.N']) * 100, 2)

62.95

### T2. Exercise / 예시문제 : 백화점고객의 1년간 데이터 (dataq 공식 예제)


In [124]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.svm import SVC
from xgboost import XGBClassifier

X_train = pd.read_csv("./data/pdq/X_train.csv", encoding="CP949")
X_test = pd.read_csv("./data/pdq/X_test.csv", encoding="CP949")
y_train = pd.read_csv("./data/pdq/y_train.csv", encoding="CP949")
ids = X_test["cust_id"]


# 데이터 확인
# X_train.info()
# X_train.describe()
# print(X_train.isnull().sum())
# print(X_test.isnull().sum())

# 데이터 전처리
X_train["환불금액"].fillna(0,inplace=True)
X_test["환불금액"].fillna(0, inplace=True)
X_train=X_train.drop(["cust_id", "주구매상품", "주구매지점"], axis=1)
X_test=X_test.drop(["cust_id", "주구매상품", "주구매지점"], axis=1)
# X_train = pd.get_dummies(X_train)
# X_test =pd.get_dummies(X_test)


mx = MinMaxScaler()
mx.fit(X_train)
st_train = mx.transform(X_train)
st_test = mx.transform(X_test)

X_tr, X_val, y_tr, y_val = train_test_split(st_train, y_train["gender"], test_size=0.2, random_state=1)

# xgb=XGBClassifier(n_estimators=500, max_depth=5, learning_rate=0.02, objective='binary:logistic')
# xgb.fit(X_tr, y_tr, early_stopping_rounds=100, eval_metric='auc', eval_set=[(X_val,y_val)], verbose=False)

# rf = RandomForestClassifier(n_estimators=300,max_depth=8,min_samples_leaf=5,random_state=1)
# rf.fit(X_tr, y_tr)

# dt = DecisionTreeClassifier(max_depth=5, random_state=1)
# dt.fit(X_tr, y_tr)

# sv = SVC()
# sv.fit(X_tr, y_tr)


 # score확인해보기

# prediction = rf.predict(X_val)

# print(accuracy_score(y_val, prediction))

# pred=rf.predict(st_test)

# submission = pd.DataFrame({"cust_id":ids, "gender":pred})
# submission.to_csv("./data/013455.csv", index=False)
# print(help(RandomForestClassifier()))


array([[0.05081002, 0.0200896 , 0.01216845, ..., 0.13729977, 0.52702703,
        0.10240964],
       [0.02296577, 0.00722639, 0.00053215, ..., 0.02371542, 0.        ,
        0.0060241 ],
       [0.02341239, 0.00652602, 0.        , ..., 0.04743083, 0.        ,
        0.0060241 ],
       ...,
       [0.0220982 , 0.00432203, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.0228559 , 0.00562554, 0.        , ..., 0.        , 0.        ,
        0.23493976],
       [0.13281818, 0.05301985, 0.01059507, ..., 0.06740171, 0.4673913 ,
        0.04819277]])

###  T2-1 titanic 예측 하기

In [59]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC 
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error


X_train = pd.read_csv("./data/titanic/train.csv")
X_test = pd.read_csv("./data/titanic/test.csv")
X_train.dropna(subset=["Embarked"], inplace=True)
X_test.dropna(subset=["Fare"], inplace=True)
# X_train.shape
ids=X_test["PassengerId"]
y_train = X_train["Survived"]


X_train.drop(["PassengerId","Ticket", "Cabin", "Name","Survived"], axis=1, inplace=True)
X_test.drop(["PassengerId","Ticket", "Cabin", "Name"], axis=1, inplace=True)

X_train["Age"].fillna(X_train["Age"].mean(), inplace=True)
X_test["Age"].fillna(X_test["Age"].mean(), inplace=True)

# # 전처리 train Age, Embarked     test Age, Fare


columns=["Embarked","Sex"]
for col in columns:
    lb=LabelEncoder()
    X_train[col]=lb.fit_transform(X_train[col])
    X_test[col]=lb.fit_transform(X_test[col])
    

# lb= LabelEncoder()
# lb.fit(X_train["Embarked"])
# X_train["Embarked"]=lb.transform(X_train["Embarked"])
# X_test["Embarked"]=lb.transform(X_test["Embarked"])
# lb.fit(X_train["Sex"])
# X_train["Sex"]=lb.transform(X_test["Sex"])
# X_test["Sex"]=lb.transform(X_test["Sex"])

mx = MinMaxScaler()
mx.fit(X_train)
X_train = mx.transform(X_train)
X_test =mx.transform(X_test)

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred= rf.predict(X_test)
print(rf.score(X_train,y_train))




0.9820022497187851


### T2-2. Pima Indians Diabetes

In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

data= pd.read_csv("./datasets/pimaid/diabetes.csv")

y = data["Outcome"]
data.drop(["Outcome"], axis=1, inplace=True)

# 데이터 전처리
# 데이터 결측값이나 이상치 확인하는 코드 작성해서 확인하고 주석 처리하기

# data.info()
# data.describe()
# data.isnull().sum()

# features=[ "Glucose", "BloodPressure", "SkinThickness", "Insulin"]
# for i in features:
#    zero = data[data[i]==0][i].count()
#    print(i, zero)

data["SkinThickness"] = data["SkinThickness"].replace(0, data["SkinThickness"].mean())
data["Insulin"] = data["Insulin"].replace(0, data["Insulin"].mean())
mm = MinMaxScaler()
mm.fit_transform(data)


X_train,X_test, y_train, y_test= train_test_split(data,y ,test_size=0.2, random_state=1)


# 의사결정나무 테스트
# dt = DecisionTreeClassifier(max_depth=5)
# dt.fit(X_train, y_train)
# y_pred = dt.predict(X_test)

# submission = pd.DataFrame({"ID":X_test.index, "Outcome":y_pred})
# print(submission)
# submission.to_csv("01234567890.csv", index=False)

rf = RandomForestClassifier(max_depth=5, min_samples_split=5)
rf.fit(X_train, y_train)
y_pred = dt.predict(X_test)
# print(y_pred)

print(f"정확도는:{accuracy_score(y_test, y_pred):.3f}")



    

정확도는:0.721


### T2-3. Adult Census Income Tutorial

In [52]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

data = pd.read_csv("./adult.csv")
# data.info()
# data.describe()
data.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
income            0
dtype: int64

### T2-5. Insurance Forecast (Regression) 

In [29]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error,r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC 

data = pd.read_csv("./data/insurance.csv")
# data.info()
# data.isnull().sum()
y=data["charges"]

# y['charges'] = np.log1p(y['charges'])

data.drop("charges", axis=1, inplace=True)


data=pd.get_dummies(data)
X_train, X_test, y_train, y_test=train_test_split(data, y, test_size=0.2, random_state=1)


# y_train['charges'] = np.log1p(y_train['charges'])

# model = DecisionTreeRegressor(max_depth=5, min_samples_split=5)
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred= model.predict(X_test)

submission = pd.DataFrame({"id":X_test.index, "charges":y_pred})
submission

MSE = mean_squared_error(y_test, y_pred)
RMSE=np.sqrt(MSE)
print(RMSE)


5956.454717976426


In [ ]:
# 9. 회귀 모델
# 9-1. 선형 회귀(릿지/라쏘)
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
model = ElasticNet()

# 9-2. 결정나무
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(random_state=0)

# 9-3. SVM
from sklearn.svm import SVR
model = SVR()

# 9-4. 랜덤포레스트
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(max_depth=10, n_estimators=100, random_state=0)

# 9-5. Ada 부스팅
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(random_state=0)

# 9-6. XG 부스팅
from xgboost import XGBClassifier
model = XGBClassifier(random_state=0)

In [50]:
### 캐글 기출 연습
X_train = pd.read_csv("./data/t2-1-train.csv")
X_test = pd.read_csv("./data/t2-1-test.csv")

import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, accuracy_score, mean_squared_error, roc_auc_score

ids=X_test["id"]
y_train=X_train["TravelInsurance"]
X_train.drop(["id","TravelInsurance"], axis=1, inplace=True)
X_test.drop(["id"], axis=1, inplace=True)
# print(X_test.shape)
columns=["Employment Type", "GraduateOrNot", "FrequentFlyer", "EverTravelledAbroad"]

#결측값 확인 "AnnualIncome"
# print(X_train.isnull().sum())
# print(X_test.isnull().sum())

X_train["AnnualIncome"]=X_train["AnnualIncome"].fillna(X_train["AnnualIncome"].mean())
X_test["AnnualIncome"]=X_test["AnnualIncome"].fillna(X_test["AnnualIncome"].mean())

# 데이터 
# 타겟값: "TravelInsurance"
# 없앨것 : "id"
# 범주형: "Employment Type", "GraduateOrNot", "FrequentFlyer", "EverTravelledAbroad"
# print(X_train.info())
# print(X_test.info())

df=pd.concat([X_train,X_test])
for cols in columns:
    lb = LabelEncoder()
    df[cols]=lb.fit_transform(df[cols])


mx=MinMaxScaler()
mx.fit(df[["AnnualIncome"]])
df["AnnualIncome"]=mx.transform(df[["AnnualIncome"]])
# print(df)

X_train=df[:X_train.shape[0]]
X_test=df[X_train.shape[0]:]

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2022)
# print(X_train.shape)
# print(y_train.shape)
# 랜덤포레스트
rf=RandomForestClassifier(n_estimators=500, max_depth=5, random_state=2022)
rf.fit(X_tr, y_tr)
pred=rf.predict_proba(X_val)
pred=pred[:,1]
# print(roc_auc_score(y_val, pred))

y_pred=rf.predict_proba(X_test)
y_pred=y_pred[:,1]

# 제출
submission = pd.DataFrame({"id":ids,"TravelInsurance":y_pred})
# print(submission.shape)
submission.to_csv("../datasets/004002519.csv", index=False)


# xgb=XGBClassifier(n_estimators=500, max_depth=5, learning_rate=0.01, objective="binary:logistic")
# xgb.fit(X_tr, y_tr, early_stopping_rounds=100, eval_metric="auc", eval_set=[X_val,y_val], verbose=False)
# pred=xgb.predict(X_val)
# print(accuracy_score(y_val, pred))


# xgb=XGBRegressor(n_estimators=300, max_depth=3, learning_rate=0.05, n_jobs=-1)
# xgb.fit(X_train, y_train, eval_set = [(X_val, y_val)], early_stopping_rounds=30, verbose=0)







(497, 2)


### 빅분기 실기 모의고사 1회차

In [51]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv')
result=(df.age//10*10).value_counts()

In [6]:
# 문제 1 마케팅 응답 고객들의 나이를 10살 단위로 변환 했을 때, 가장 많은 인원을 가진 나이대는? (0~9 : 0 , 10~19 : 10)
result=(df.age//10*10).value_counts().index[0]
print(result)

30


In [54]:
# 문제 2 가장 많은 나이대 구간의 인원은 몇명인가?
result=(df.age//10*10).value_counts()

print(result.iloc[1])

3198


In [12]:
# 나이가 25살 이상 29살 미만인 응답 고객들중 housing컬럼의 값이 yes인 고객의 수는?
result=df[(df["age"]>=25)&(df["age"]<29)&(df["housing"]=="yes")].shape[0]
print(result)

504


In [13]:
# 4번문제 numeric한 값을 가지지 않은 컬럼들중 unique한 값을 가장 많이 가지는 컬럼은?
lst= [] 
for col in df.select_dtypes(exclude='int'):
    target = df[col]
    lst.append([col,target.nunique()])

result = pd.DataFrame(lst).sort_values(1,ascending=False).values[0][0]
print(result)

job


In [16]:
# 5번 balance 컬럼값들의 평균값 이상을 가지는 데이터를 ID값을 기준으로 내림차순 정렬했을때 상위 100개 데이터의 balance값의 평균은?
means=df["balance"].mean()
result=df[df["balance"]>=means].sort_values(by="ID", ascending=False).iloc[:100]
results=result["balance"].mean()
print(results)

3473.73


In [28]:
# 6번 가장 많은 광고를 집행했던 날짜는 언제인가? (데이터 그대로 일(숫자),달(영문)으로 표기)
result = df[['day','month']].value_counts().index[0]
print(result)

(15, 'may')


In [29]:
# 7번 데이터의 job이 unknown 상태인 고객들의 age 컬럼 값의 정규성을 검정하고자 한다. 샤피로 검정의 p-value값을 구하여라
from scipy.stats import shapiro
result = shapiro(df[df.job =='unknown'].age)[1]
print(result)

0.1961131989955902


In [47]:
# 8번 age와 balance의 상관계수를 구하여라
cor=df[["age","balance"]].corr().iloc[0,1]
print(cor)

0.10198734763981462


In [48]:
df

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y
0,13829,29,technician,single,tertiary,no,18254,no,no,cellular,11,may,2,-1,0,unknown,no
1,22677,26,services,single,secondary,no,512,yes,yes,unknown,5,jun,3,-1,0,unknown,no
2,10541,30,management,single,secondary,no,135,no,no,cellular,14,aug,2,-1,0,unknown,no
3,13689,41,technician,married,unknown,no,30,yes,no,cellular,10,jul,1,-1,0,unknown,no
4,11304,27,admin.,single,secondary,no,321,no,yes,unknown,2,sep,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12865,14023,47,technician,married,secondary,no,1167,yes,no,cellular,30,apr,1,87,5,failure,yes
12866,17259,31,unknown,married,secondary,no,111,no,no,cellular,21,nov,2,93,2,failure,yes
12867,15200,37,unemployed,single,tertiary,no,1316,yes,no,cellular,18,nov,1,172,2,failure,no
12868,13775,42,management,married,tertiary,no,479,yes,no,unknown,28,may,2,-1,0,unknown,no


In [49]:
#  10번 각 job에 따라 divorced/married 인원의 비율을 확인 했을 때 그 값이 가장 높은 값은?
t = df.groupby(['job','marital']).size().reset_index()
pivotdf = t.pivot_table(index='job',columns='marital')[0]
pivotdf = pivotdf.fillna(0)
pivotdf['ratio'] = pivotdf['divorced'] / pivotdf['married']

result = pivotdf.sort_values('ratio').ratio.values[-1]
print(result)

0.2831050228310502


### 회귀 문제 연습

In [27]:

import pandas as pd
X_train = pd.read_csv("./data/t2-2-X_train.csv")
X_test= pd.read_csv("./data/t2-2-X_test.csv")
y_train = pd.read_csv("./data/t2-2-y_train.csv")


import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error, confusion_matrix, mean_absolute_error

ids=X_test["Serial No."]
# 데이터 확인
# print(X_train.info())
# print(X_test.info())
# print(y_train)
# print(X_train.isnull().sum())
# print(X_test.isnull().sum())

X_train=X_train.drop(["Serial No."], axis=1)
X_test=X_test.drop(["Serial No."], axis=1)
y_train=y_train.drop(["Serial No."], axis=1)

cols=X_train.columns

st= StandardScaler()
st.fit(X_train)
X_train_st=st.transform(X_train)

st.fit(X_test)
X_test_st=st.transform(X_test)

X_train=pd.DataFrame(X_train_st,columns=cols)
X_test=pd.DataFrame(X_test_st, columns=cols)

X_tr,X_val, y_tr, y_val = train_test_split(X_train,y_train, test_size=0.2, random_state=2022)

# print(X_train.shape)
# print(X_test.shape)

# rf = RandomForestRegressor(n_estimators=500, max_depth=10, random_state=2022)
# rf.fit(X_tr,y_tr)
# pred=rf.predict(X_val)
# print(r2_score(y_val,pred))

# y_pred=rf.predict(X_test)


# DecisionTreeRegressor
# dc = DecisionTreeRegressor(max_depth=10, random_state=2022)
# dc.fit(X_tr, y_tr)
# pred=dc.predict(X_val)
# print(r2_score(y_val, pred))


import xgboost as xgb
from sklearn.metrics import r2_score
xgb_model = xgb.XGBRegressor(n_estimators=300, max_depth=3, learning_rate=0.05, n_jobs=-1)
xgb_model.fit(X_train, y_train, eval_set = [(X_val, y_val)], early_stopping_rounds=30, verbose=0)
pred=xgb_model.predict(X_val)
print(r2_score(y_val, pred))


# submission = pd.DataFrame({"Serial No.":ids,"chance of Admit":y_pred})
# submission.to_csv("004002519.csv", index=False)


0.9425721695222399


In [32]:
# 작업형 2유형 문제
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error
import datetime as dt
import numpy as np



train= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv')
test= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/test.csv')
train["y"]=train["y"].apply(lambda x: 1 if(x=="yes") else 0)

# 데이터 확인
# train.info()
# test.info()
y=train["y"]
ids=test["ID"]


# print(test.isnull().sum())
train.drop(["ID","day","month","y"], axis=1, inplace=True)
test.drop(["ID","day","month"], axis=1, inplace=True)
df=pd.concat([train, test])
colums=["job", "marital","education","default","housing","loan","contact","poutcome"]
for cols in colums:
    lb = LabelEncoder()
    df[cols]=lb.fit_transform(df[cols])
mx=MinMaxScaler()
mx.fit(df[["pdays"]])
df["pdays"]=mx.transform(df[["pdays"]])

train=df.iloc[:len(train)]
test=df.iloc[len(train):]

X_train, X_val, y_train,y_val =train_test_split(train,y,test_size=0.2, random_state=2022)


rf = RandomForestClassifier(random_state=2022, max_depth=10, n_estimators=500)
rf.fit(X_train, y_train)


pred=rf.predict_proba(X_val)
pred=pred[:,1]
print(roc_auc_score(y_val, pred))



# y_pred=rf.predict_proba(test)
# y_pred=y_pred[:,1]


# submission = pd.DataFrame({"id":ids,"y":y_pred})
# submission.to_csv("./data/004002519.csv", index=False)





0.7515004974024538


### 빅분기 실기 모의고사 2회차